In [511]:
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
def clean_transaction_description(description):
    cib_ref_number_regex = r"^FT.{12}$"
    generic_ref_regex = r"\w*\d\w*[a-zA-Z]\w*|\w*[a-zA-Z]\w*\d\w*"
    numbers_regex = r"\d+"
    single_letter_regex = r"\b\w\b"
    opening_balance = "OPENING BALANCE"
    closing_balance = "CLOSING BALANCE"

    text = description.replace("\BNK", "")
    text = text.replace(opening_balance, "")
    text = text.replace(closing_balance, "")
    text = re.sub(cib_ref_number_regex, '', text)
    text = re.sub(generic_ref_regex, '', text)
    text = re.sub(numbers_regex, '', text)
    text = re.sub(single_letter_regex, '', text)
    text = text.replace("-", "")
    text = re.sub("\s\s+" , " ", text)
    return text.strip()


In [3]:
def filter_instapay_ach_data(description):
    ipn_regex = r"Payee Name:"
    cib_online_transfer = "Online Transfer"
    ach_regax = r'\bACH\b'
    credit_interest = r"Credit Interest"
    funds_regex = r'Funds \.\.\.'  
    quaterly_interest = r"Statement Quarterly fees"
    
    if re.match(ipn_regex, description):
        return True
    
    if re.match(cib_online_transfer, description):
        return True
    
    if re.match(ach_regax, description):
        return True
    
    if re.match(credit_interest, description):
        return True

    if re.match(funds_regex, description):
        return True

    if re.match(quaterly_interest, description):
        return True
    
    return False 

In [512]:
def convert_written_date_to_number(date_str):
    # Define mapping of month abbreviations to month numbers
    month_map = {
        'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4,
        'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8,
        'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
    }
    
    # Extract day, month abbreviation, and year from date string
    day = int(date_str[:2])
    month_abbr = date_str[2:5]
    year = int('20' + date_str[5:])
    
    # Convert month abbreviation to month number
    month_num = month_map[month_abbr]
    
    # Create a datetime object with the extracted values
    date = datetime(year, month_num, day)
    
    return date.strftime('%d/%m/%Y')

In [513]:
file_name = "9"

In [514]:
df = pd.read_csv(f"../bank_statements/raw/{file_name}.csv")

In [515]:
df.head()

,Date,Txn Date,Narrative,Credit,Debit
0,25-03,25-03,CASH BACK CAIRO,69.0,NaN
1,10-03,10-03,MCDONALDS MAADI 1 C03 CAIRO N. -07A,NaN,161.00
2,11-03,11-03,ADIDAS - One Kattameya FO CAIRO N. -07A,NaN,999.00
3,11-03,11-03,ELDAHAN CAIRO,NaN,566.47
4,12-03,12-03,ADIDAS - One Kattameya FO CAIRO N. -07A,999.0,NaN


In [516]:
df["Debit"] = df["Debit"].replace(",", "", regex=True)
df["Credit"] = df["Credit"].replace(",", "", regex=True)


In [517]:
df.head()

,Date,Txn Date,Narrative,Credit,Debit
0,25-03,25-03,CASH BACK CAIRO,69.0,NaN
1,10-03,10-03,MCDONALDS MAADI 1 C03 CAIRO N. -07A,NaN,161.00
2,11-03,11-03,ADIDAS - One Kattameya FO CAIRO N. -07A,NaN,999.00
3,11-03,11-03,ELDAHAN CAIRO,NaN,566.47
4,12-03,12-03,ADIDAS - One Kattameya FO CAIRO N. -07A,999.0,NaN


In [518]:
df["Debit"] = pd.to_numeric(df["Debit"], errors="coerce")
df["Credit"] = pd.to_numeric(df["Credit"], errors="coerce")

In [519]:
df["Value"] = df["Debit"].apply(lambda x: (x*-1) if x is not np.nan else None)

In [520]:
non_values = df.loc[df["Value"].isnull()]

In [521]:
non_values["Value"] = non_values["Credit"].apply(lambda x: (x) if x is not np.nan else None)

/var/folders/n8/fx58pk156kzbywrs2435cgw80000gn/T/ipykernel_2356/2981624320.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_values["Value"] = non_values["Credit"].apply(lambda x: (x) if x is not np.nan else None)


In [522]:
df.loc[non_values.index] = non_values

In [523]:
df.drop(["Debit", "Credit"], axis=1, inplace=True)

In [524]:
df.head()

,Date,Txn Date,Narrative,Value
0,25-03,25-03,CASH BACK CAIRO,69.00
1,10-03,10-03,MCDONALDS MAADI 1 C03 CAIRO N. -07A,-161.00
2,11-03,11-03,ADIDAS - One Kattameya FO CAIRO N. -07A,-999.00
3,11-03,11-03,ELDAHAN CAIRO,-566.47
4,12-03,12-03,ADIDAS - One Kattameya FO CAIRO N. -07A,999.00


In [525]:
df["Clean_Narrative"] = df["Narrative"].apply(clean_transaction_description)

In [526]:
df.head()

,Date,Txn Date,Narrative,Value,Clean_Narrative
0,25-03,25-03,CASH BACK CAIRO,69.00,CASH BACK CAIRO
1,10-03,10-03,MCDONALDS MAADI 1 C03 CAIRO N. -07A,-161.00,MCDONALDS MAADI CAIRO .
2,11-03,11-03,ADIDAS - One Kattameya FO CAIRO N. -07A,-999.00,ADIDAS One Kattameya FO CAIRO .
3,11-03,11-03,ELDAHAN CAIRO,-566.47,ELDAHAN CAIRO
4,12-03,12-03,ADIDAS - One Kattameya FO CAIRO N. -07A,999.00,ADIDAS One Kattameya FO CAIRO .


In [527]:
df["Clean_Narrative"] = df["Clean_Narrative"].apply(lambda x: np.nan if x == "" else x)

In [528]:
df.dropna(inplace=True, subset=["Clean_Narrative"])

In [529]:
df["Date"] = df["Date"].apply(convert_written_date_to_number)
df["Txn Date"] = df["Txn Date"].apply(convert_written_date_to_number)

KeyError: '-03'

In [ ]:
df.to_csv(f"../bank_statements/pre_processed/{file_name}.csv", index=False)